In [ ]:
#import packages
import trimesh
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#mesh an stl file and load it
mesh = trimesh.load_mesh('./final-stls/cube.stl')
mesh;

In [ ]:
##use this to convert scenes to meshes
"""
def as_mesh(scene_or_mesh):
    #convert a possible scene to a mesh.
    #if conversion occurs, the returned mesh has only vertex and face data.
    if isinstance(scene_or_mesh, trimesh.Scene):
        if len(scene_or_mesh.geometry) == 0:
            mesh = None  # empty scene
        else:
            # we lose texture information here
            mesh = trimesh.util.concatenate(
                tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                    for g in scene_or_mesh.geometry.values()))
    else:
        assert(isinstance(mesh, trimesh.Trimesh))
        mesh = scene_or_mesh
    return mesh
mesh = as_mesh(mesh)
"""

In [ ]:
#check mesh bounds
mesh.bounds;

In [ ]:
#check mesh faces
mesh.faces.shape;

In [ ]:
#check mesh vertices
mesh.vertices;

In [ ]:
#check mesh normals
mesh.vertex_normals;

In [ ]:
#remesh with m value
m = 10
new_mesh = trimesh.Trimesh(*trimesh.remesh.subdivide_to_size(mesh.vertices, mesh.faces, m))
new_mesh;

In [ ]:
#export the new mesh
new_mesh.export('./final-stls/new-cube-m10.stl');

In [ ]:
#check new mesh bounds
new_mesh.bounds;

In [ ]:
#check new mesh faces
new_mesh.faces.shape;

In [ ]:
#check new mesh vertices
new_mesh.vertices;


In [ ]:
#check a range of normals
new_mesh.vertex_normals[100:120];

In [ ]:
#save coords and normals
np.save('./testing/input/coords.npy', new_mesh.vertices)
vn = np.array(new_mesh.vertex_normals, order = 'C')#change column to row major
np.save('./testing/input/normals.npy', vn)

In [ ]:
#check coords
np.load('./testing/input/coords.npy');

In [ ]:
#check normals
np.load('./testing/input/normals.npy');

In [ ]:
#run masbcpp with r
r = 0.21
!/tmp/stuff/masbcpp/compute_ma -r $r -p 0 -d 0 ./testing/input ./testing/output;#denoising deactivated

In [ ]:
#check inner MA coords
ma_coords_in = np.load('./testing/output/ma_coords_in.npy')
ma_coords_in;

In [ ]:
#check a range of inner MA coords
ma_coords_in[10:100];

In [ ]:
#create a point cloud using inner MA coords
ma_coords_in_pc = trimesh.PointCloud(ma_coords_in)
ma_coords_in_pc;

In [ ]:
#load inner MA radii
ma_rad_in = np.load('./testing/output/ma_rad_in.npy')
ma_rad_in;

In [ ]:
#remove negative values from inner MA radii
ma_rad_in[ma_rad_in>0];

In [ ]:
#remove non-finite inner MA radii
"""
dirty = ma_rad_in[ma_rad_in>0]
clean = dirty[dirty<1E100]
#load a range of inner radii values
plt.hist(clean, 50)
plt.ylabel('Number of spheres')
plt.xlabel('Radius, r (mm)')
plt.title('Brake Rotor Fillet Rim: Number of Spheres Versus Radius, r (mm)')
"""

In [ ]:
#plot histogram of number of spheres against radii at inner MA points
plt.hist(ma_rad_in[ma_rad_in>0],50)
plt.ylabel('Number of spheres')
plt.xlabel('Radius, r (mm)')
plt.title('Cube: Number of Spheres Versus Radius, r (mm)')

In [ ]:
#remove NaNs from inner MA coords
in_no_nans = np.nan_to_num(ma_coords_in)
in_no_nans;

In [ ]:
#check inner MA coords have no NaNs
in_no_nans[10:150];

In [ ]:
#create a point cloud using inner MA coords with no NaNs
ma_coords_in_no_nans = trimesh.PointCloud(in_no_nans)
ma_coords_in_no_nans

In [ ]:
#export the inner MA coords wtih no NaNs as .obj file for MeshLab viewing 
ma_coords_in_no_nans.export('./testing/output/ma_coords_in_no_nans.obj');

In [ ]:
#load inner MA coord indexes
ma_qidx_in = np.load('./testing/output/ma_qidx_in.npy')
ma_qidx_in;